In [51]:
# biblioteki
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
import numpy as np

Wykorzystany zbiór: https://sites.google.com/view/danish-fungi-dataset

In [2]:
# Ścieżki do plików CSV i folderu ze zdjęciami
metadata_train_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M-metadata/DF20M-train_metadata_PROD.csv'
metadata_test_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M-metadata/DF20M-public_test_metadata_PROD.csv'
images_folder_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M'

# Wczytanie metadanych
metadata_train = pd.read_csv(metadata_train_path)
metadata_test = pd.read_csv(metadata_test_path)

In [3]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Gid,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,DNK.5.14_1,Nordfyns,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,DNK.1.28_1,Rudersdal,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,DNK.1.20_1,Hillerød,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,DNK.1.11_1,Frederikssund,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,DNK.3.1_1,Aalborg,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG


In [4]:
metadata_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3640 entries, 0 to 3639
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gbifID                3640 non-null   int64  
 1   eventDate             3639 non-null   object 
 2   year                  3639 non-null   float64
 3   month                 3639 non-null   float64
 4   day                   3639 non-null   float64
 5   countryCode           3640 non-null   object 
 6   locality              3636 non-null   object 
 7   taxonID               3640 non-null   float64
 8   scientificName        3640 non-null   object 
 9   kingdom               3640 non-null   object 
 10  phylum                3640 non-null   object 
 11  class                 3640 non-null   object 
 12  order                 3640 non-null   object 
 13  family                3640 non-null   object 
 14  genus                 3640 non-null   object 
 15  specificEpithet      

In [5]:
# Upewnij się, że kolumna 'image_path' jest w obu ramkach danych
metadata_train['full_image_path'] = metadata_train['image_path'].apply(lambda x: os.path.join(images_folder_path, x))
metadata_test['full_image_path'] = metadata_test['image_path'].apply(lambda x: os.path.join(images_folder_path, x))

In [6]:
metadata_train['full_image_path'][0]

'/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M/2862684394-136762.JPG'

In [7]:
metadata_test.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path
0,2238504270,2017-08-20T00:00:00,2017.0,8.0,20.0,DK,Store Dyrehave,20020.0,Russula nigricans (Bull.) Fr.,Fungi,...,Hillerød,2238504270-171705,soil,Dan Ingemann Jensen,55.905585,12.362367,75.0,Deciduous woodland,2238504270-171705.JPG,/Users/milenabiernacka/Desktop/studia/magister...
1,2238511676,2017-09-17T00:00:00,2017.0,9.0,17.0,DK,Store Hareskov,17238.0,Mycena haematopus (Pers.) P.Kumm.,Fungi,...,Furesø,2238511676-246956,dead wood (including bark),Tarquin Netherway,55.769675,12.401108,15.0,Deciduous woodland,2238511676-246956.JPG,/Users/milenabiernacka/Desktop/studia/magister...
2,2883271536,2020-10-18T00:00:00,2020.0,10.0,18.0,DK,Lillerød,17289.0,Mycena rosea (Schumach.) Gramberg,Fungi,...,Allerød,2883271536-291057,soil,Dan Ingemann Jensen,55.878946,12.359130,40.0,Deciduous woodland,2883271536-291057.JPG,/Users/milenabiernacka/Desktop/studia/magister...
3,2446759186,2019-10-26T00:00:00,2019.0,10.0,26.0,DK,Ollerup,10079.0,Agaricus impudicus (Rea) Pilát,Fungi,...,Svendborg,2446759186-271681,soil,Dan Schou,55.068441,10.515358,50.0,Unmanaged deciduous woodland,2446759186-271681.JPG,/Users/milenabiernacka/Desktop/studia/magister...
4,2382321410,2019-08-30T00:00:00,2019.0,8.0,30.0,DK,"Dyrehave, Valdemars Slot (inkl Pederskov)",19939.0,Russula chloroides (Krombh.) Bres.,Fungi,...,Svendborg,2382321410-191132,soil,Kirsten Bjørnsson,55.013310,10.644585,75.0,Deciduous woodland,2382321410-191132.JPG,/Users/milenabiernacka/Desktop/studia/magister...


In [8]:
metadata_train['scientificName'].value_counts()

scientificName
Mycena galericulata (Scop.) Gray       1099
Clitocybe nebularis (Batsch) Quél.     1003
Amanita muscaria (L.) Lam., 1783        863
Boletus edulis Bull.                    811
Amanita rubescens (Pers.) Gray          713
                                       ... 
Agaricus bisporus (J.E.Lange) Pilat      33
Russula laeta Jul.Schäff.                32
Clitocybe diatreta (Fr.) P.Kumm.         32
Clitocybe nitrophila Bon                 29
Russula insignis Quél.                   28
Name: count, Length: 182, dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder

# Zakodowanie etykiet
label_encoder = LabelEncoder()
metadata_train['class_encoded'] = label_encoder.fit_transform(metadata_train['scientificName'])
metadata_test['class_encoded'] = label_encoder.transform(metadata_test['scientificName'])  # Uwaga: używamy transform, nie fit_transform

In [10]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path,class_encoded
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG,/Users/milenabiernacka/Desktop/studia/magister...,70
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG,/Users/milenabiernacka/Desktop/studia/magister...,1
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG,/Users/milenabiernacka/Desktop/studia/magister...,151
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG,/Users/milenabiernacka/Desktop/studia/magister...,40
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG,/Users/milenabiernacka/Desktop/studia/magister...,89


In [11]:
# Definicja ImageDataGeneratorów
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path,class_encoded
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG,/Users/milenabiernacka/Desktop/studia/magister...,70
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG,/Users/milenabiernacka/Desktop/studia/magister...,1
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG,/Users/milenabiernacka/Desktop/studia/magister...,151
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG,/Users/milenabiernacka/Desktop/studia/magister...,40
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG,/Users/milenabiernacka/Desktop/studia/magister...,89


In [15]:
metadata_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32753 entries, 0 to 32752
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gbifID                32753 non-null  int64  
 1   eventDate             32748 non-null  object 
 2   year                  32748 non-null  float64
 3   month                 32748 non-null  float64
 4   day                   32748 non-null  float64
 5   countryCode           32753 non-null  object 
 6   locality              32737 non-null  object 
 7   taxonID               32753 non-null  float64
 8   scientificName        32753 non-null  object 
 9   kingdom               32753 non-null  object 
 10  phylum                32753 non-null  object 
 11  class                 32753 non-null  object 
 12  order                 32753 non-null  object 
 13  family                32753 non-null  object 
 14  genus                 32753 non-null  object 
 15  specificEpithet    

In [16]:
# Utwórz teraz zmniejszone próbki danych
train_sample = metadata_train.sample(frac=0.2, random_state=42) 
test_sample = metadata_test.sample(frac=0.2, random_state=42) 

# Lista wszystkich unikalnych klas z oryginalnego DataFrame
all_classes = sorted(metadata_train['scientificName'].unique().tolist())

# Tworzenie generatorów z jawnie określoną listą klas
train_generator_sample = train_datagen.flow_from_dataframe(
    dataframe=train_sample,
    x_col='full_image_path',
    y_col='scientificName',
    target_size=(224, 224),
    batch_size=32,
    classes=all_classes,  # Zapewnij listę wszystkich klas
    class_mode='categorical')

test_generator_sample = test_datagen.flow_from_dataframe(
    dataframe=test_sample,
    x_col='full_image_path',
    y_col='scientificName',
    target_size=(224, 224),
    batch_size=32,
    classes=all_classes,  # Zapewnij listę wszystkich klas
    class_mode='categorical')


Found 6551 validated image filenames belonging to 182 classes.
Found 728 validated image filenames belonging to 182 classes.


In [17]:
# Wczytanie MobileNetV2 bez górnych warstw
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie górnych warstw
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)  # liczba neuronów odpowiada liczbie klas

# Zbudowanie całego modelu
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilacja modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trening modelu
model.fit(train_generator_sample, validation_data=test_generator_sample, epochs=10, steps_per_epoch=train_generator_sample.samples // train_generator_sample.batch_size,
          validation_steps=len(test_generator_sample))

Epoch 1/10
204/204 [==============================] - 290s 1s/step - loss: 4.3618 - accuracy: 0.1088 - val_loss: 18.8836 - val_accuracy: 0.0096
Epoch 2/10
204/204 [==============================] - 290s 1s/step - loss: 3.5540 - accuracy: 0.1902 - val_loss: 17.1339 - val_accuracy: 0.0165
Epoch 3/10
204/204 [==============================] - 295s 1s/step - loss: 3.0987 - accuracy: 0.2533 - val_loss: 16.3753 - val_accuracy: 0.0082
Epoch 4/10
204/204 [==============================] - 302s 1s/step - loss: 2.7222 - accuracy: 0.3069 - val_loss: 27.6825 - val_accuracy: 0.0124
Epoch 5/10
204/204 [==============================] - 296s 1s/step - loss: 2.4315 - accuracy: 0.3576 - val_loss: 9.5334 - val_accuracy: 0.0234
Epoch 6/10
204/204 [==============================] - 325s 2s/step - loss: 2.1972 - accuracy: 0.4094 - val_loss: 10.2197 - val_accuracy: 0.0302
Epoch 7/10
204/204 [==============================] - 354s 2s/step - loss: 1.9539 - accuracy: 0.4617 - val_loss: 10.3734 - val_accuracy: 

In [18]:
test_loss, test_accuracy = model.evaluate(test_generator_sample, steps=test_generator_sample.samples // test_generator_sample.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

22/22 [==============================] - 15s 693ms/step - loss: 11.3220 - accuracy: 0.0170
Test Loss: 11.322044372558594, Test Accuracy: 0.017045455053448677


In [19]:
model.evaluate(test_generator_sample)

23/23 [==============================] - 16s 696ms/step - loss: 11.2848 - accuracy: 0.0179


[11.284760475158691, 0.01785714365541935]

In [1]:
print("hi")

hi


# Danish Fungi Validation Set

In [75]:
metadata_path = '/Users/milenabiernacka/Desktop/praca-magisterska/DF/validation/FungiCLEF2023_val_metadata_PRODUCTION.csv'
photos_path = '/Users/milenabiernacka/Desktop/praca-magisterska/DF/validation/DF21_300'

metadata = pd.read_csv(metadata_path)
metadata.head()

,observationID,month,day,countryCode,locality,taxonID,scientificName,kingdom,phylum,class,...,Substrate,Latitude,Longitude,CoorUncert,Habitat,image_path,filename,MetaSubstrate,class_id,poisonous
0,3008834344,1.0,1.0,DK,Kildevænget,15233,Hypoxylon fragiforme (Pers.) J.Kickx f.,Fungi,Ascomycota,Sordariomycetes,...,NaN,55.946963,12.292050,31.0,Deciduous woodland,0-3008834344.JPG,0-3008834344,wood,702,0
1,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,...,NaN,55.947665,12.287160,16.0,coniferous woodland/plantation,0-3008834345.JPG,0-3008834345,wood,553,0
2,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,...,NaN,55.947665,12.287160,16.0,coniferous woodland/plantation,1-3008834345.JPG,1-3008834345,wood,553,0
3,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,...,mosses,56.529817,9.550255,30.0,coniferous woodland/plantation,0-3008834353.JPG,0-3008834353,mosses,267,0
4,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,...,mosses,56.529817,9.550255,30.0,coniferous woodland/plantation,1-3008834353.JPG,1-3008834353,mosses,267,0


In [76]:
metadata['full_image_path'] = metadata['image_path'].apply(lambda x: os.path.join(photos_path, x))

In [77]:
metadata.head()

,observationID,month,day,countryCode,locality,taxonID,scientificName,kingdom,phylum,class,...,Latitude,Longitude,CoorUncert,Habitat,image_path,filename,MetaSubstrate,class_id,poisonous,full_image_path
0,3008834344,1.0,1.0,DK,Kildevænget,15233,Hypoxylon fragiforme (Pers.) J.Kickx f.,Fungi,Ascomycota,Sordariomycetes,...,55.946963,12.292050,31.0,Deciduous woodland,0-3008834344.JPG,0-3008834344,wood,702,0,/Users/milenabiernacka/Desktop/praca-magisters...
1,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,...,55.947665,12.287160,16.0,coniferous woodland/plantation,0-3008834345.JPG,0-3008834345,wood,553,0,/Users/milenabiernacka/Desktop/praca-magisters...
2,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,...,55.947665,12.287160,16.0,coniferous woodland/plantation,1-3008834345.JPG,1-3008834345,wood,553,0,/Users/milenabiernacka/Desktop/praca-magisters...
3,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,...,56.529817,9.550255,30.0,coniferous woodland/plantation,0-3008834353.JPG,0-3008834353,mosses,267,0,/Users/milenabiernacka/Desktop/praca-magisters...
4,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,...,56.529817,9.550255,30.0,coniferous woodland/plantation,1-3008834353.JPG,1-3008834353,mosses,267,0,/Users/milenabiernacka/Desktop/praca-magisters...


In [78]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60832 entries, 0 to 60831
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   observationID    60832 non-null  int64  
 1   month            60832 non-null  float64
 2   day              60832 non-null  float64
 3   countryCode      60832 non-null  object 
 4   locality         60813 non-null  object 
 5   taxonID          60832 non-null  int64  
 6   scientificName   60832 non-null  object 
 7   kingdom          60832 non-null  object 
 8   phylum           60832 non-null  object 
 9   class            60787 non-null  object 
 10  order            60664 non-null  object 
 11  family           60394 non-null  object 
 12  genus            60832 non-null  object 
 13  specificEpithet  60832 non-null  object 
 14  taxonRank        60832 non-null  object 
 15  species          60832 non-null  object 
 16  level0Gid        60639 non-null  object 
 17  level0Name  

In [79]:
half_metadata = metadata.sample(frac=0.5, random_state=42)

In [80]:
half_metadata.head()

,observationID,month,day,countryCode,locality,taxonID,scientificName,kingdom,phylum,class,...,Latitude,Longitude,CoorUncert,Habitat,image_path,filename,MetaSubstrate,class_id,poisonous,full_image_path
48791,3052841406,3.0,3.0,DK,Filsø Gårde,13952,Exidia glandulosa (Bull.) Fr.,Fungi,Basidiomycota,Agaricomycetes,...,56.245372,8.412633,6.0,Unmanaged deciduous woodland,0-3052841406.JPG,0-3052841406,wood,468,0,/Users/milenabiernacka/Desktop/praca-magisters...
14384,3386480353,10.0,12.0,DK,Overlund,15850,Lactarius glyciosmus (Fr.) Fr.,Fungi,Basidiomycota,Agaricomycetes,...,56.458939,9.441329,28.0,natural grassland,0-3386480353.JPG,0-3386480353,jord,774,0,/Users/milenabiernacka/Desktop/praca-magisters...
11713,3399806330,10.0,30.0,DK,Kulebjerg Overdrev,16660,Macrolepiota procera (Scop.) Singer,Fungi,Basidiomycota,Agaricomycetes,...,55.543602,11.439074,25.0,natural grassland,1-3399806330.JPG,1-3399806330,jord,-1,0,/Users/milenabiernacka/Desktop/praca-magisters...
3803,3091023316,4.0,22.0,DK,Greve Strand,69190,Gemmina gemmarum (Boud.) Raitv.,Fungi,Ascomycota,Leotiomycetes,...,55.593711,12.294706,65.0,Thorny scrubland,5-3091023316.JPG,5-3091023316,plant material,-1,0,/Users/milenabiernacka/Desktop/praca-magisters...
37397,3336954336,8.0,18.0,DK,Ahl Plantage,15908,Lactarius quietus (Fr.) Fr.,Fungi,Basidiomycota,Agaricomycetes,...,56.179937,10.640115,10.0,bog,1-3336954336.JPG,1-3336954336,jord,787,0,/Users/milenabiernacka/Desktop/praca-magisters...


In [81]:
half_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30416 entries, 48791 to 2918
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   observationID    30416 non-null  int64  
 1   month            30416 non-null  float64
 2   day              30416 non-null  float64
 3   countryCode      30416 non-null  object 
 4   locality         30410 non-null  object 
 5   taxonID          30416 non-null  int64  
 6   scientificName   30416 non-null  object 
 7   kingdom          30416 non-null  object 
 8   phylum           30416 non-null  object 
 9   class            30387 non-null  object 
 10  order            30319 non-null  object 
 11  family           30175 non-null  object 
 12  genus            30416 non-null  object 
 13  specificEpithet  30416 non-null  object 
 14  taxonRank        30416 non-null  object 
 15  species          30416 non-null  object 
 16  level0Gid        30313 non-null  object 
 17  level0Name    

In [82]:
data = half_metadata[['species', 'full_image_path', 'image_path', 'poisonous']]
data.head()

,species,full_image_path,image_path,poisonous
48791,Exidia glandulosa,/Users/milenabiernacka/Desktop/praca-magisters...,0-3052841406.JPG,0
14384,Lactarius glyciosmus,/Users/milenabiernacka/Desktop/praca-magisters...,0-3386480353.JPG,0
11713,Macrolepiota procera,/Users/milenabiernacka/Desktop/praca-magisters...,1-3399806330.JPG,0
3803,Gemmina gemmarum,/Users/milenabiernacka/Desktop/praca-magisters...,5-3091023316.JPG,0
37397,Lactarius quietus,/Users/milenabiernacka/Desktop/praca-magisters...,1-3336954336.JPG,0


In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30416 entries, 48791 to 2918
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   species          30416 non-null  object
 1   full_image_path  30416 non-null  object
 2   image_path       30416 non-null  object
 3   poisonous        30416 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [84]:
print(data['species'].value_counts())

species
Micarea lignaria              245
Trametes versicolor           226
Fomitopsis pinicola           180
Stereum hirsutum              154
Auricularia auricula-judae    150
                             ... 
Enterographa crassa             1
Cortinarius turgidus            1
Pluteus eludens                 1
Clathrus ruber                  1
Cortinarius limonius            1
Name: count, Length: 2432, dtype: int64


In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

In [44]:
X_train

,species,full_image_path,image_path,poisonous
55032,Anthostomella phaeosticta,/Users/milenabiernacka/Desktop/praca-magisters...,4-3382565302.JPG,0
27368,Xanthoria parietina,/Users/milenabiernacka/Desktop/praca-magisters...,0-3052918330.JPG,0
23338,Entoloma rhodopolium,/Users/milenabiernacka/Desktop/praca-magisters...,1-3406489302.JPG,1
33821,Hyaloperonospora dentariae,/Users/milenabiernacka/Desktop/praca-magisters...,0-3109983325.JPG,0
27486,Phacidium lauri,/Users/milenabiernacka/Desktop/praca-magisters...,0-3068185325.JPG,0
...,...,...,...,...
56113,Laetiporus sulphureus,/Users/milenabiernacka/Desktop/praca-magisters...,1-3343235330.JPG,0
38144,Mycena rosea,/Users/milenabiernacka/Desktop/praca-magisters...,0-3382568339.JPG,1
1165,Podosphaera balsaminae,/Users/milenabiernacka/Desktop/praca-magisters...,1-3329732343.JPG,0
26906,Bjerkandera fumosa,/Users/milenabiernacka/Desktop/praca-magisters...,0-3008835353.JPG,0


In [45]:
X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=42)

In [46]:
X_val

,species,full_image_path,image_path,poisonous
46348,Lentinus substrictus,/Users/milenabiernacka/Desktop/praca-magisters...,1-3131220315.JPG,0
14106,Gyromitra infula,/Users/milenabiernacka/Desktop/praca-magisters...,2-3358348499.JPG,0
30994,Cerioporus squamosus,/Users/milenabiernacka/Desktop/praca-magisters...,4-3320496348.JPG,0
28837,Tricholoma lascivum,/Users/milenabiernacka/Desktop/praca-magisters...,1-3382571311.JPG,0
47846,Hygrocybe coccinea,/Users/milenabiernacka/Desktop/praca-magisters...,1-3392608405.JPG,0
...,...,...,...,...
39784,Auriscalpium vulgare,/Users/milenabiernacka/Desktop/praca-magisters...,0-3080227328.JPG,0
55418,Lecidella scabra,/Users/milenabiernacka/Desktop/praca-magisters...,0-3122828331.JPG,0
33619,Melanelixia subaurifera,/Users/milenabiernacka/Desktop/praca-magisters...,2-3059588329.JPG,0
11237,Inocybe whitei,/Users/milenabiernacka/Desktop/praca-magisters...,0-3382572430.JPG,0


In [57]:
X_train['species'].nunique()

2228

In [87]:
# Podział danych na zbiór treningowy i pozostałe dane (testowy + walidacyjny)
train_data, test_val_data = train_test_split(data, test_size=0.4, random_state=42)

# Podział pozostałych danych na zbiór testowy i walidacyjny
test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)

In [88]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ścieżka do zdjęć - upewnij się, że jest poprawna
photos_path = '/Users/milenabiernacka/Desktop/praca-magisterska/DF/DF21_300'

# Utwórz generator obrazów z augmentacją danych dla zbioru treningowego
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Utwórz generator obrazów bez augmentacji danych dla zbioru walidacyjnego i testowego
datagen_test_val = ImageDataGenerator(
    rescale=1./255
)

# Przygotowanie generatorów
train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_data,
    directory=photos_path,
    x_col='full_image_path',
    y_col='species',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=True
)

test_generator = datagen_test_val.flow_from_dataframe(
    dataframe=test_data,
    directory=photos_path,
    x_col='full_image_path',
    y_col='species',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=False
)

val_generator = datagen_test_val.flow_from_dataframe(
    dataframe=val_data,
    directory=photos_path,
    x_col='full_image_path',
    y_col='species',
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=False
)


Found 18249 validated image filenames belonging to 2195 classes.
Found 6083 validated image filenames belonging to 1551 classes.
Found 6084 validated image filenames belonging to 1530 classes.


In [89]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Ładowanie MobileNetV2 bez górnych warstw
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie górnych warstw
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# Liczba klas (gatunków) do przewidywania
n_classes = len(train_generator.class_indices)
predictions = Dense(n_classes, activation='softmax')(x)

# Zbudowanie modelu
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilacja modelu
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [90]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

Epoch 1/10


/Users/milenabiernacka/Desktop/praca-magisterska/Mushroom-classification/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


571/571 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0269 - loss: 7.0268

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1530), output.shape=(None, 2195)

In [91]:
len(train_generator.class_indices)

2195

In [92]:
len(test_generator.class_indices)

1551

In [93]:
len(val_generator.class_indices)

1530

In [94]:
# Zliczenie wystąpień każdej klasy
class_counts = data['species'].value_counts()

# Filtracja klas, które mają więcej niż jedno wystąpienie
data_filtered = data[data['species'].isin(class_counts[class_counts > 1].index)].copy()

# Mapowanie nazw klas na indeksy klas
class_to_index = {c: idx for idx, c in enumerate(data_filtered['species'].unique())}
index_to_class = {v: k for k, v in class_to_index.items()}

# Zamiana nazw klas na indeksy
data_filtered['species'] = data_filtered['species'].map(class_to_index)

In [96]:
data_filtered['species'].value_counts()

species
60      245
280     226
6       180
16      154
104     150
       ... 
1535      2
1527      2
1524      2
1523      2
2037      2
Name: count, Length: 2038, dtype: int64